In [1]:
url = "http://www.nasdaq.com/earnings/earnings-calendar.aspx"
frame = '<iframe src=' + url + 'width=700 height=500></iframe>'

In [2]:
from bs4 import BeautifulSoup
import urllib
r = urllib.urlopen(url).read()
soup = BeautifulSoup(r)
print type(soup)

<class 'bs4.BeautifulSoup'>


C:\ProgramData\Anaconda2\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file C:\ProgramData\Anaconda2\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [3]:
print soup.prettify()[0:1000]

<!DOCTYPE html>
<html class="inner no-js" lang="en-us" xmlns:fb="https://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
 <head>
  <!-- Google Tag Manager -->
  <script>
   (function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
})(window,document,'script','dataLayer','GTM-K2BQVP7');
  </script>
  <!-- End Google Tag Manager -->
  <script>
   (function(){
var is_chrome;
if(typeof navigator.vendor!="undefined")
	is_chrome = ((navigator.userAgent.toLowerCase().indexOf('chrome') > -1) &&(navigator.vendor.toLowerCase().indexOf("google") > -1));
else
	is_chrome = false;
if(is_chrome)
{
//instart
//Copyright Intart Logic Thu Nov 16 2017 - version: 8.3.21
!function(t){if(function(){var n=!0;try{var i="object"==typeof t.IXC_320_8513696329873064


In [4]:
companiesTable = soup.find(id='ECCompaniesTable')
print(type(companiesTable))

<class 'bs4.element.Tag'>


In [5]:
from enum import Enum
class Hours(Enum):
    DAY = 1
    NIGHT = 2
    UNKNOWN = 3

In [6]:
rows = companiesTable.findChildren(['th', 'tr'])

In [7]:
def parse_time_col(time_child):
    if len(time_child.select('a')) == 0:
        return Hours.UNKNOWN
    
    title = time_child.select('a')[0]['title']
    if title == 'After Hours Quotes':
        return Hours.NIGHT
    if title == 'Pre-market Quotes':
        return Hours.DAY
    return Hours.UNKNOWN

In [8]:
from re import sub
from decimal import Decimal

def parse_decimal(money):
    if money == '$n/a':
        return None
    return Decimal(money[1:])

In [9]:
import pandas as pd
from datetime import datetime

def parse_table():
    column_titles = ['Time', 'Symbol', 'Reported Date', 'Quarter', 'ConsensusEPS', '# of Ests', 'Unknown', 'LastYearEPS', 'PercentSurprise']
    df = pd.DataFrame(columns=column_titles)

    row_marker = 0
    for row in rows:
        children = row.findChildren('td')
        if len(children) == 0:
            continue
            
        row_data = [column.get_text().strip() for column in children]
        row_data[0] = parse_time_col(children[0])
        row_data[2] = datetime.strptime(row_data[2], '%m/%d/%Y')
        row_data[4] = parse_decimal(row_data[4])
        row_data[5] = int(row_data[5])
        if row_data[6] == 'n/a':
            row_data[6] = None
        else:
            row_data[6] = datetime.strptime(row_data[6], '%m/%d/%Y')
        row_data[7] = parse_decimal(row_data[7])
            
        df.loc[row_marker] = row_data
        row_marker += 1

    return df

In [10]:
df = parse_table()
df.tail()

,Time,Symbol,Reported Date,Quarter,ConsensusEPS,# of Ests,Unknown,LastYearEPS,PercentSurprise
15,Hours.NIGHT,The Descartes Systems Group Inc. (DSGX) Market...,2017-11-29,Oct 2017,0.1,4,2016-11-30 00:00:00,0.08,Met
16,Hours.DAY,Tiffany & Co. (TIF) Market Cap: $11.62B,2017-11-29,Oct 2017,0.76,10,2016-11-29 00:00:00,0.76,Met
17,Hours.NIGHT,"Tilly's, Inc. (TLYS) Market Cap: $369.22M",2017-11-29,Oct 2017,0.21,3,2016-11-30 00:00:00,0.22,Met
18,Hours.NIGHT,"Workday, Inc. (WDAY) Market Cap: $23.91B",2017-11-29,Oct 2017,-0.42,32,2016-12-01 00:00:00,-0.5,Met
19,Hours.DAY,Yintech Investment Holdings Limited (YIN) Mark...,2017-11-29,Sep 2017,None,0,2016-11-15 00:00:00,0.56,Met


In [11]:
filtered_df = df[(df['Time'] == Hours.NIGHT) & (df['ConsensusEPS'] > df['LastYearEPS'])]
# filtered_df = df[(df['Time'] == Hours.DAY) & (df['ConsensusEPS'] > df['LastYearEPS'])]
filtered_df

,Time,Symbol,Reported Date,Quarter,ConsensusEPS,# of Ests,Unknown,LastYearEPS,PercentSurprise
0,Hours.NIGHT,At Home Group Inc. (HOME) Market Cap: $1.31B,2017-11-29,Oct 2017,0.04,5,2016-12-06 00:00:00,0.03,Met
2,Hours.NIGHT,"Box, Inc. (BOX) Market Cap: $3.08B",2017-11-29,Oct 2017,-0.29,5,2016-11-30 00:00:00,-0.3,Met
5,Hours.NIGHT,La-Z-Boy Incorporated (LZB) Market Cap: $1.35B,2017-11-29,Oct 2017,0.46,3,2016-11-30 00:00:00,0.42,Met
6,Hours.NIGHT,Monmouth Real Estate Investment Corporation (M...,2017-11-29,Sep 2017,0.21,3,2016-11-28 00:00:00,0.18,Met
10,Hours.NIGHT,PVH Corp. (PVH) Market Cap: $10.47B,2017-11-29,Oct 2017,2.91,7,2016-11-30 00:00:00,2.6,Met
13,Hours.NIGHT,Semtech Corporation (SMTC) Market Cap: $2.57B,2017-11-29,Oct 2017,0.34,1,2016-11-30 00:00:00,0.28,Met
15,Hours.NIGHT,The Descartes Systems Group Inc. (DSGX) Market...,2017-11-29,Oct 2017,0.1,4,2016-11-30 00:00:00,0.08,Met
18,Hours.NIGHT,"Workday, Inc. (WDAY) Market Cap: $23.91B",2017-11-29,Oct 2017,-0.42,32,2016-12-01 00:00:00,-0.5,Met


In [12]:
symbols = [s[s.find("(")+1:s.find(")")] for s in filtered_df['Symbol']]
print len(symbols)

8


In [13]:
symbols

[u'HOME', u'BOX', u'LZB', u'MNR', u'PVH', u'SMTC', u'DSGX', u'WDAY']

In [14]:
filtered_df.describe()

,Time,Symbol,Reported Date,Quarter,ConsensusEPS,# of Ests,Unknown,LastYearEPS,PercentSurprise
count,8,8,8,8,8,8.0,8,8,8
unique,1,8,1,2,8,6.0,4,8,1
top,Hours.NIGHT,"Box, Inc. (BOX) Market Cap: $3.08B",2017-11-29 00:00:00,Oct 2017,-0.42,5.0,2016-11-30 00:00:00,0.28,Met
freq,8,1,8,7,1,2.0,5,1,8
first,NaN,NaN,2017-11-29 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,2017-11-29 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
s = ["hello"]
print s

['hello']
